# ## second clustering approach: pair of keywords 
since that I use a new app dataset (that is the union of the Official Samsung SmartThings: https://github.com/vomar18/SmartThingsPublic and the nsslabcuus: https://github.com/nsslabcuus/IoTMon)
1. check every single app description in order to identify:"The entity keyword with the highest aggregated value is considered as a representative keyword for the cluster"
2. create the similarity matrix
3. save the pair of word with the highest aggregation value (over than 90%??)
4. save the result inside: "cluster_pysical_channels.JSON"

In [2]:
# !!attention!! use python 3.6 for manage the packet stanza that is 
# the Official Stanford NLP Python Library for Many Human Languages

from gensim.models import Word2Vec, KeyedVectors 
import json   # for read the JSON file
import stanza # Official Stanford NLP Python Library for Many Human Languages
import numpy as np

In [3]:
#read only descriptions and save them
descriptions = {} #dictionary with descriptions
with open("app-description.JSON", "r") as read_file: # load the JSON file with app descriptions
    descriptions = json.load(read_file) 

In [4]:
print("total app description found: "+ str(len(descriptions)))

total app description found: 140


In [5]:
stanza.download('en')       # This downloads the English models for the neural pipeline
nlp = stanza.Pipeline('en') # This sets up a default neural pipeline in English
model = KeyedVectors.load_word2vec_format('/home/volta/Documenti/CCS18_CS/word2vec/GoogleNews-vectors-negative300.bin', binary=True, limit=100000) # load word2vec pre-trained

2021-09-29 09:54:52 INFO: Downloading default packages for language: en (English)...
2021-09-29 09:54:53 INFO: File exists: /home/volta/stanza_resources/en/default.zip.
2021-09-29 09:54:58 INFO: Finished downloading models and saved to /home/volta/stanza_resources.
2021-09-29 09:54:58 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-09-29 09:54:58 INFO: Use device: cpu
2021-09-29 09:54:58 INFO: Loading: tokenize
2021-09-29 09:54:58 INFO: Loading: pos
2021-09-29 09:54:59 INFO: Loading: lemma
2021-09-29 09:54:59 INFO: Loading: depparse
2021-09-29 09:54:59 INFO: Loading: sentiment
2021-09-29 09:54:59 INFO: Loading: ner
2021-09-29 09:55:00 INFO: Done loading processors!


In [6]:
set_of_keywords = []        # set of most similar keywords extrapolated from every single app description
# the keywords accepted are the NN and NNS as the example of cap 4.2-p835
POS_accepted=["NN", "NNS"]  # part of speech tags accepted ---> deleted: "NNP", "NNPS"
percentage_lv = 15          # level of similarity accepted between entity keywords of the same app description
                            # this level came from the analysis of the descriptions of example application of table 1

for app in descriptions.keys(): # check every app descriptions
    keywords = []       # keywords found inside app description
    keyword_vector = {} # dictionary keyword -> vector rappresentation (just for check if word2vec can 
                        # calculate the similarity between NN and NNS kwywords)
    print("analysing app description: ",app)
    des_analysed = nlp(descriptions[app]) # apply Stanford NLP to description
    for sent in des_analysed.sentences: 
        for word in sent.words:           # check each word of the description
            if word.xpos in POS_accepted: # check is it is a NN or NNS
                keywords_found = word.text
                if not keywords_found in keywords: # if not already saved that keyword 
                    #print(f"word:{word.text}")
                    keywords.append(keywords_found) # save it
    
    for i in range(len(keywords)): # for every keywords saved from the description 
            try:
                word = str(keywords[i])
                if len(word) > 1:                      # if is not a single letter or symbol
                    keyword_vector[word] = model[word] # try to use Word2Vec
                else:
                    print("word not processed: ",word)
            except KeyError as err:
                print("handling Error: ", err) # some words can not be processed by Word2Vec
    
    keywords = list(keyword_vector.keys()) # save all keyword that can be processed by word2vec
    tot_keyword_found = len(keywords)
    
    if tot_keyword_found == 0:
        print("no one keyword found")
    
    elif tot_keyword_found == 1:
        print("tot_keywords:1 -", keywords[0])
        if not keywords[0] in set_of_keywords:
            set_of_keywords.append(keywords[0]) # just a single keyword inside a description -> save it
    
    elif tot_keyword_found == 2: # find only a pair of keywords so directly save them
        print("tot_keywords:2 -", keywords)
        if not keywords[0] in set_of_keywords:
            set_of_keywords.append(keywords[0])
        if not keywords[1] in set_of_keywords:
            set_of_keywords.append(keywords[1])
    else: # find a lot of kwywords --> save only the most similar
        
        similarity_matrix = np.ones((tot_keyword_found, tot_keyword_found))
        
        print("tot_keywords:",tot_keyword_found,"-", keywords)
        for key_x in range(tot_keyword_found):
            for key_y in range(key_x, tot_keyword_found):
                if key_x != key_y:
                    #print("check",keywords[key_x], " and ", keywords[key_y])
                    # -------------------------------  model.similarity() is equal to 1 - cosine.distance() --------
                    similarity_matrix[key_x][key_y] = (model.similarity(keywords[key_x], keywords[key_y])) * 100
                    if similarity_matrix[key_x][key_y] >= percentage_lv: # for every pair of keywords save only
                        if not keywords[key_x] in set_of_keywords:       # the most similar
                            print("add", keywords[key_x])
                            set_of_keywords.append(keywords[key_x])
                        if not keywords[key_y] in set_of_keywords:
                            print("add", keywords[key_y])
                            set_of_keywords.append(keywords[key_y])

print(len(set_of_keywords))
set_of_keywords.sort()
set_of_keywords

analysing app description:  alfred-workflow


/home/volta/.local/lib/python3.6/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


tot_keywords:2 - ['things', 'graph']
analysing app description:  beacon-control
tot_keywords: 5 - ['Home', 'phrase', 'lights', 'door', 'region']
add lights
add door
analysing app description:  big-turn-off
tot_keywords:1 - lights
analysing app description:  big-turn-on
tot_keywords:1 - lights
analysing app description:  bon-voyage
tot_keywords: 14 - ['set', 'Presence', 'tags', 'smartphones', 'mode', 'change', 'everyone', 'conjunction', 'Turn', 'lights', 'appliances', 'thermostat', 'security', 'apps']
add set
add change
add Presence
add Turn
add security
add tags
add apps
add smartphones
add appliances
add mode
add thermostat
add everyone
analysing app description:  bose-soundtouch-control
word not processed:  ®
tot_keywords: 4 - ['Sound', 'actions', 'place', 'home']
add actions
add place
add home
analysing app description:  brighten-dark-places
tot_keywords: 3 - ['lights', 'sensor', 'space']
add sensor
add space
analysing app description:  brighten-my-path
tot_keywords:2 - ['lights', '

tot_keywords: 16 - ['devices', 'capabilities', 'app', 'ambient', 'light', 'notification', 'drawer', 'cabinet', 'meds', 'reminder', 'time', 'day', 'draw', 'minutes', 'message', 'LED']
add capabilities
add ambient
add meds
add LED
add drawer
add reminder
analysing app description:  medicine-reminder
tot_keywords: 8 - ['reminder', 'medicine', 'cabinet', 'drawer', 'time', 'notification', 'text', 'message']
add medicine
analysing app description:  mini-hue-controller
tot_keywords:2 - ['Hue', 'bulbs']
analysing app description:  mood-cube
tot_keywords:2 - ['lighting', 'cube']
analysing app description:  nfc-tag-toggle
handling Error:  "Key 'toggling' not present"
tot_keywords: 7 - ['switch', 'lock', 'garage', 'door', 'Tag', 'touch', 'event']
add touch
add Tag
analysing app description:  notify-me-when
tot_keywords: 3 - ['notifications', 'anything', 'home']
analysing app description:  notify-me-when-it-opens
tot_keywords: 4 - ['push', 'message', 'phone', 'sensor']
add phone
analysing app desc

handling Error:  "Key 'Ubi' not present"
handling Error:  "Key 'SmartThings' not present"
tot_keywords:2 - ['device', 'Account']
analysing app description:  undead-early-warning
tot_keywords:1 - Warning
analysing app description:  unlock-it-when-i-arrive
tot_keywords:2 - ['door', 'location']
analysing app description:  virtual-device-creator
tot_keywords:1 - devices
analysing app description:  virtual-thermostat
tot_keywords: 8 - ['space', 'heater', 'window', 'air_conditioner', 'conjunction', 'temperature', 'sensor', 'Multi']
add air_conditioner
add conjunction
analysing app description:  wattvision-manager
tot_keywords: 4 - ['house', 'energy', 'use', 'account']
analysing app description:  weatherbug-home
tot_keywords:1 - Weather
analysing app description:  weather-underground-pws-connect
tot_keywords:1 - sensor
analysing app description:  wemo-connect
handling Error:  "Key 'WeMo' not present"
handling Error:  "Key 'Wemo' not present"
tot_keywords: 3 - ['Switch', 'Motion', 'sensor']
ad

['API',
 'Account',
 'Alarm',
 'Button',
 'Change',
 'Changes',
 'Device',
 'Devices',
 'Harmony',
 'Hue',
 'LED',
 'Manager',
 'Monitors',
 'Motion',
 'Multi',
 'Note',
 'Panic',
 'Presence',
 'SMS',
 'Service',
 'Setup',
 'Switch',
 'TV',
 'Tag',
 'Turn',
 'Updates',
 'Users',
 'Warning',
 'Weather',
 'acceleration',
 'account',
 'action',
 'actions',
 'activities',
 'air_conditioner',
 'alarm',
 'ambient',
 'amount',
 'anytime',
 'app',
 'appliance',
 'appliances',
 'apps',
 'area',
 'arrival',
 'bathroom',
 'battery',
 'bed',
 'bedroom',
 'brightness',
 'buddies',
 'bulbs',
 'burst',
 'button',
 'buttons',
 'cabinet',
 'camera',
 'cameras',
 'candle',
 'capabilities',
 'car',
 'carbon_monoxide',
 'carpooling',
 'change',
 'changes',
 'chat',
 'color',
 'colors',
 'conjunction',
 'contact',
 'controller',
 'controls',
 'cube',
 'day',
 'days',
 'departure',
 'device',
 'devices',
 'dogs',
 'door',
 'doors',
 'drawer',
 'element',
 'endpoint',
 'energy',
 'environment',
 'events',
 '

In [7]:
print(len(set_of_keywords))
set_of_keywords

214


['API',
 'Account',
 'Alarm',
 'Button',
 'Change',
 'Changes',
 'Device',
 'Devices',
 'Harmony',
 'Hue',
 'LED',
 'Manager',
 'Monitors',
 'Motion',
 'Multi',
 'Note',
 'Panic',
 'Presence',
 'SMS',
 'Service',
 'Setup',
 'Switch',
 'TV',
 'Tag',
 'Turn',
 'Updates',
 'Users',
 'Warning',
 'Weather',
 'acceleration',
 'account',
 'action',
 'actions',
 'activities',
 'air_conditioner',
 'alarm',
 'ambient',
 'amount',
 'anytime',
 'app',
 'appliance',
 'appliances',
 'apps',
 'area',
 'arrival',
 'bathroom',
 'battery',
 'bed',
 'bedroom',
 'brightness',
 'buddies',
 'bulbs',
 'burst',
 'button',
 'buttons',
 'cabinet',
 'camera',
 'cameras',
 'candle',
 'capabilities',
 'car',
 'carbon_monoxide',
 'carpooling',
 'change',
 'changes',
 'chat',
 'color',
 'colors',
 'conjunction',
 'contact',
 'controller',
 'controls',
 'cube',
 'day',
 'days',
 'departure',
 'device',
 'devices',
 'dogs',
 'door',
 'doors',
 'drawer',
 'element',
 'endpoint',
 'energy',
 'environment',
 'events',
 '

In [8]:
tot_keywords = len(set_of_keywords) # set of most similar keywords 
physical_channels_matrix = np.zeros((tot_keywords ,tot_keywords ))
for key_x in range(tot_keywords):
    for key_y in range(tot_keywords):
        #print("check",keywords[key_x], " and ", keywords[key_y])
        physical_channels_matrix[key_x][key_y] = model.similarity(set_of_keywords[key_x], set_of_keywords[key_y])

physical_channels_matrix = physical_channels_matrix * 100        
np.set_printoptions(precision=2)
print(physical_channels_matrix)

[[100.    21.94   8.18 ...  14.65  15.26   4.84]
 [ 21.94 100.    13.33 ...   7.85  -1.06   3.5 ]
 [  8.18  13.33 100.   ...  11.98  11.77   6.11]
 ...
 [ 14.65   7.85  11.98 ... 100.    72.61   3.31]
 [ 15.26  -1.06  11.77 ...  72.61 100.    11.16]
 [  4.84   3.5    6.11 ...   3.31  11.16 100.  ]]


In [13]:
tot_keywords = len(set_of_keywords)
percentage_channel_level = 30 # percentage (%) of similarity accepted for the creation of a keyword cluster
physical_channel = {} # dictionary made of [keyword] : list of most similar keywords associated with the [keyword]

for i in range(tot_keywords): # create an empty dictionary
    physical_channel[set_of_keywords[i]]=[]

for key_x in range(tot_keywords):
    for key_y in range(tot_keywords): # now for the cluster save only the most similar keywords
        if (key_y != key_x) and (physical_channels_matrix[key_x][key_y] >= percentage_channel_level):
            physical_channel[set_of_keywords[key_x]].append(set_of_keywords[key_y])
            
physical_channel

{'API': ['Device', 'app', 'apps', 'notifications', 'user'],
 'Account': ['account'],
 'Alarm': ['Device',
  'Monitors',
  'Panic',
  'Warning',
  'alarm',
  'sirens',
  'sound'],
 'Button': ['Turn', 'button', 'buttons'],
 'Change': ['Changes', 'change', 'changes'],
 'Changes': ['Change', 'Updates', 'change', 'changes'],
 'Device': ['API',
  'Alarm',
  'Devices',
  'LED',
  'Monitors',
  'Multi',
  'Presence',
  'Setup',
  'Tag',
  'app',
  'apps',
  'device',
  'devices',
  'endpoint',
  'sensor',
  'sensors',
  'user'],
 'Devices': ['Device',
  'LED',
  'Monitors',
  'device',
  'devices',
  'sensor',
  'sensors',
  'smartphones'],
 'Harmony': [],
 'Hue': [],
 'LED': ['Device',
  'Devices',
  'Monitors',
  'air_conditioner',
  'ambient',
  'battery',
  'brightness',
  'bulbs',
  'candle',
  'color',
  'device',
  'light',
  'lighting',
  'lights',
  'sensor',
  'sensors',
  'switches'],
 'Manager': [],
 'Monitors': ['Alarm', 'Device', 'Devices', 'LED', 'sensors'],
 'Motion': ['motion'

In [14]:
# physical channel identification (table 1 - p835)
dict_physical_channel = {}

temp = physical_channel["temperature"] # save all the keywords of a cluster 
temp.append("temperature") # don't forget to add also the main keyword 
dict_physical_channel["temperature"] = temp # save them inside a dictionary - temperature

temp = physical_channel["humidity"]
temp.append("humidity")
dict_physical_channel["humidity"]= temp # humidity

temp =physical_channel["lights"]
temp.append("lights")
dict_physical_channel["illumination"]=temp # illumination

temp = physical_channel["motion"]
temp.append("motion")
dict_physical_channel["motion"]=temp # motion

temp = physical_channel["location"]
temp.append("location")
dict_physical_channel["location"]=temp # location

temp = physical_channel["carbon_monoxide"]
temp.append("carbon_monoxide")
dict_physical_channel["smoke"]=temp # smoke

temp=physical_channel["messages"]
temp.append("messages")
dict_physical_channel["leakage"]=temp # leakage

temp=physical_channel["time"]
temp.append("time")
dict_physical_channel["time"]=temp # time

temp = ['location', 'mode']
dict_physical_channel["locationMode"]=temp # locationMode

temp=physical_channel["lock"]
temp.append("lock")
dict_physical_channel["lock"]=temp # lock

dict_physical_channel

{'temperature': ['air_conditioner',
  'ambient',
  'brightness',
  'carbon_monoxide',
  'heater',
  'humidity',
  'moisture',
  'sensor',
  'sensors',
  'thermostat',
  'ventilation',
  'water',
  'weather',
  'temperature'],
 'humidity': ['Weather',
  'air_conditioner',
  'ambient',
  'brightness',
  'carbon_monoxide',
  'moisture',
  'temperature',
  'thermostat',
  'ventilation',
  'weather',
  'humidity'],
 'illumination': ['LED',
  'air_conditioner',
  'alarm',
  'appliances',
  'brightness',
  'bulbs',
  'camera',
  'cameras',
  'candle',
  'colors',
  'heater',
  'light',
  'lighting',
  'sirens',
  'switch',
  'switches',
  'thermostat',
  'lights'],
 'motion': ['Motion', 'action', 'motion'],
 'location': ['area', 'place', 'location'],
 'smoke': ['air_conditioner',
  'ambient',
  'fire',
  'heater',
  'humidity',
  'leaks',
  'moisture',
  'temperature',
  'thermostat',
  'ventilation',
  'carbon_monoxide'],
 'leakage': ['SMS',
  'chat',
  'mail',
  'mailbox',
  'message',
  'n

In [15]:
with open("cluster_pysical_channels.JSON","w") as outfile:
    json.dump(dict_physical_channel, outfile)